### Free space

No use of symmetry factors, just use all elements
        
        fullME.py

In [1]:
from sympy import *
import numpy as np
import re
from joblib import Parallel, delayed

### Definitions and auxiliary functions

In [2]:
xij, yij, zij, b, eta = symbols("xij yij zij b eta", real=True) #probably need FH and VH symbols too

# indices for summation:
## Caution, cannot use lambda, instead use Lambda
## do not use eta as index, it's the viscosity
alpha, beta, gamma, gamma1, gamma2, gamma3, gamma4, kappa, kappa1, kappa2, mu, nu, nu1, nu2, Lambda, eta2, zeta, sigma = symbols(
    'alpha, beta, gamma, gamma1, gamma2, gamma3, gamma4, kappa, kappa1, kappa2, mu, nu, nu1, nu2, Lambda, eta2, zeta, sigma', integer=True)


## for FH and VH checkout the 'Indexed' package to define them as FH[1], FH[2] etc.

rij = Array([xij, yij, zij])

def norm(rij):
    return sqrt(summation(rij[alpha]**2,(alpha,0,2)))

KroneckerDelta = eye(3)

def Laplacian(G):
    return diff(G, rij[0], rij[0]) + diff(G, rij[1], rij[1]) + diff(G, rij[2], rij[2])

def eps(alpha, beta, gamma):
    return LeviCivita(alpha,beta,gamma)

In [3]:
def Curl(alpha, beta): ## the function to take the curl of and the two remaining indices 
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(alpha,i,j)*diff(G(j,beta),rij[i]) ## G is symmetric in its indices
    return result

def CurlCurl(alpha, beta): ## the function to take the curl of and the two remaining indices 
    result=0
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    result += eps(alpha,i,j)*eps(beta,k,l)*diff(G(j,l), rij[i], rij[k])
    return result

### Matrix elements

In [4]:
def G(alpha, beta):
    return 1/(8*pi*eta)*(KroneckerDelta[alpha,beta]/norm(rij) + rij[alpha]*rij[beta]/norm(rij)**3)

def delG(alpha, beta, gamma):
    return 1/(8*pi*eta)*((-KroneckerDelta[alpha,beta]*rij[gamma] + KroneckerDelta[alpha,gamma]*rij[beta] + KroneckerDelta[beta,gamma]*rij[alpha])/norm(rij)**3
                         - 3*rij[alpha]*rij[beta]*rij[gamma]/norm(rij)**5)

def lapG(alpha, beta):
    return 1/(4*pi*eta)*(KroneckerDelta[alpha,beta]/norm(rij)**3 - 3*rij[alpha]*rij[beta]/norm(rij)**5)

In [5]:
##auxiliary functions
def dellapG(alpha, beta, kappa1):
    return diff(lapG(alpha, beta), rij[kappa1])

def deldelG(zeta, beta, kappa1, nu):
    return diff(delG(zeta, beta, kappa1), rij[nu])

def deldellapG(alpha, beta, gamma1, kappa1):
    return diff(dellapG(alpha,beta, gamma1), rij[kappa1])

def deldeldelG(alpha, beta, gamma1, gamma2, gamma3):
    return diff(deldelG(alpha, beta, gamma1, gamma2), rij[gamma3])

def deldeldeldelG(alpha, beta, gamma1, gamma2, gamma3, gamma4):
    return diff(deldeldelG(alpha, beta, gamma1, gamma2, gamma3), rij[gamma4])

In [6]:
# GLL
def G1s1s(alpha, beta):
    return simplify(simplify(G(alpha, beta) + b**2/3*lapG(alpha, beta)))

def G1s2a(alpha, beta):
    return simplify(-0.5*b*Curl(beta, alpha))
#summation(summation(eps(beta, nu, zeta)*delG(alpha, zeta, nu), (nu,0,2)), (zeta,0,2))

def G2a1s(alpha, beta):
    return simplify(simplify(b*Curl(alpha,beta)))
#summation(summation(eps(alpha,nu,zeta)*delG(zeta,beta,nu), (nu,0,2)), (zeta,0,2))

def G2a2a(alpha, beta):
    return simplify(-0.5*b*b*CurlCurl(alpha,beta))
#summation(summation(summation(summation(eps(alpha,mu,kappa)*eps(beta,nu,zeta)*deldelG(kappa,zeta,nu,mu),(mu,0,2)), (kappa,0,2)), (nu,0,2)), (zeta,0,2))

In [7]:
## GL2s and G2sL
def G1s2s(alpha, kappa1, beta):
    return simplify(-0.5*b*((delG(alpha, beta, kappa1) + delG(alpha, kappa1, beta)) + 4*b**2/15*(dellapG(alpha, beta, kappa1) + dellapG(alpha, kappa1, beta))))

def G2a2s(alpha, kappa1, beta):
    return simplify(-0.5*b*b*(diff(Curl(alpha, beta),rij[kappa1]) + diff(Curl(alpha, kappa1),rij[beta])))
#summation(summation(eps(alpha,nu,zeta)*(deldelG(zeta, beta, kappa1, nu) + deldelG(zeta, kappa1, beta, nu)), (nu,0,2)), (zeta,0,2))

def G2s1s(alpha, gamma1, beta):
    return simplify(0.5*b*((delG(alpha, beta, gamma1) + delG(gamma1, beta, alpha)) + 4*b**2/15*(dellapG(alpha,beta,gamma1) + dellapG(gamma1, beta, alpha))))

def G2s2a(alpha, gamma1, mu):
    return simplify(-0.25*b*b*(diff(Curl(mu,gamma1), rij[alpha]) + diff(Curl(mu,alpha), rij[gamma1]))) #sign change from -> eps(mu, kappa1, beta)
#summation(summation(eps(beta, kappa1, mu)*(deldelG(gamma1, beta, alpha, kappa1) + deldelG(alpha, beta, gamma1, kappa1)), (kappa1, 0, 2)), (beta,0,2))


## G2s2s
def G2s2s(alpha, gamma1, kappa1, beta):
    return simplify(-0.25*b*b*((deldelG(alpha, beta, gamma1, kappa1) + deldelG(gamma1, beta, alpha, kappa1)) + 
                      (deldelG(alpha,kappa1, gamma1, beta) + deldelG(gamma1, kappa1, alpha, beta)) +
                      b**2/5*(deldellapG(alpha,beta,gamma1,kappa1) + deldellapG(gamma1,beta,alpha,kappa1)) + 
                      b**2/5*(deldellapG(alpha, kappa1, gamma1, beta) + deldellapG(gamma1,kappa1,alpha,beta))))


## higher matrix elements
def G1s3t(alpha, beta):
    return simplify(-1/5*b**2*lapG(alpha,beta))


# symmetry for matrix elements of form l sigma, 3s
def g13sSym(alpha,kappa2,kappa1,beta):
    return 1/3*(deldelG(alpha,beta,kappa1,kappa2) + deldelG(alpha,kappa2,kappa1,beta) + deldelG(alpha,kappa1,beta,kappa2)) - 1/15*(
        KroneckerDelta[kappa1,kappa2]*lapG(alpha,beta) + KroneckerDelta[kappa1,beta]*lapG(alpha,kappa2) + KroneckerDelta[beta,kappa2]*lapG(alpha,kappa1))

def G1s3s(alpha,kappa2, kappa1, beta):
    return simplify(b**2*(g13sSym(alpha,kappa2,kappa1,beta) + 5*b**2/21*Laplacian(g13sSym(alpha,kappa2,kappa1,beta))))


# symmetry of matrix elements of form 3s, l sigma
def g3s1Sym(alpha,gamma1,gamma2,beta):
    return 1/3*(deldelG(alpha,beta,gamma1,gamma2) + deldelG(gamma1,beta,gamma2,alpha) + deldelG(gamma2,beta,alpha,gamma1)) - 1/15*(
        KroneckerDelta[gamma1, gamma2]*lapG(alpha,beta) + KroneckerDelta[gamma2,alpha]*lapG(gamma1,beta) + KroneckerDelta[alpha,gamma1]*lapG(gamma2,beta))

def G3s1s(alpha,gamma1,gamma2,beta):
    return simplify(b**2*(g3s1Sym(alpha,gamma1,gamma2,beta) + 5*b**2/21*Laplacian(g3s1Sym(alpha,gamma1,gamma2,beta))))

def G3a1s(Lambda, kappa2, beta):
    return simplify(0.5*b**2*(diff(Curl(Lambda, beta), rij[kappa2]) + diff(Curl(kappa2,beta), rij[Lambda])))
#summation(summation(eps(Lambda, gamma1, alpha)*deldelG(alpha,beta,gamma1,kappa2) +
#                    eps(kappa2, gamma1, alpha)*deldelG(alpha,beta,gamma1,Lambda), (gamma1,0,2)), (alpha,0,2))

def G1s3a(alpha, nu2, sigma):
    return simplify(2/3*b**2*(diff(Curl(sigma, alpha),rij[nu2]) - 1/3*summation(eps(sigma,nu2,beta)*lapG(alpha,beta), (beta,0,2)))) # sign change due to --> eps(sigma,nu1,beta)
#summation(summation(eps(beta,nu1,sigma)*(deldelG(alpha,beta,nu1,nu2) - 1/3*KroneckerDelta[nu1,nu2]*lapG(alpha,beta)), (beta,0,2)), (nu1,0,2))

def g3sCurlSym(alpha,gamma1,gamma2,mu):
    return 1/3*(diff(Curl(mu,alpha), rij[gamma1],rij[gamma2]) + diff(Curl(mu,gamma1), rij[gamma2],rij[alpha]) + diff(Curl(mu,gamma2), rij[alpha],rij[gamma1])) 
#- 1/15*(KroneckerDelta[gamma1, gamma2]*Laplacian(Curl(mu,alpha)) +KroneckerDelta[gamma2, alpha]*Laplacian(Curl(mu,gamma1)) +
#      KroneckerDelta[alpha, gamma1]*Laplacian(Curl(mu,gamma2)))    ##this part is zero anyway
        

def G3s2a(alpha,gamma1,gamma2,mu):
    return simplify(-0.5*b**3*g3sCurlSym(alpha, gamma1, gamma2, mu))## sign change due to -> eps(mu, kappa1, beta)
#0.5*b^3*Sum[Sum[eps[beta, kappa1, mu]*D[g3s1Sym[alpha, gamma1, gamma2, beta], rij[kappa1]], {kappa1, 3}], {beta, 3}]

def G3a2a(Lambda, kappa2, mu):
    return simplify(-0.25*b**3*(diff(CurlCurl(mu,Lambda), rij[kappa2]) + diff(CurlCurl(mu,kappa2), rij[Lambda]))) #sign change -> eps(mu,kappa1,beta)


def k2s2sSym(alpha,gamma1,kappa1,beta):
    return deldelG(alpha,kappa1,gamma1,beta) + deldelG(alpha,beta,gamma1,kappa1) + deldelG(gamma1,kappa1,alpha,beta) + deldelG(gamma1,beta,alpha,kappa1)

def K2s2s(alpha,gamma1,kappa1,beta):
    return simplify(-2*pi*eta*b**3/3*(k2s2sSym(alpha,gamma1,kappa1,beta) + b**2/5*Laplacian(k2s2sSym(alpha,gamma1,kappa1,beta))))


def k3s2sSym(alpha,gamma1,gamma2,beta,gamma):
    return diff(g3s1Sym(alpha,gamma1,gamma2,gamma), rij[beta]) + diff(g3s1Sym(alpha,gamma1,gamma2,beta), rij[gamma])

def K3s2s(alpha,gamma1,gamma2,beta,gamma):
    return simplify(-4*pi*eta*b**4/3*(k3s2sSym(alpha,gamma1,gamma2,beta,gamma) + 6*b**2/35*Laplacian(k3s2sSym(alpha,gamma1,gamma2,beta,gamma))))


def K3a2s(Lambda, kappa2, beta, gamma):
    return simplify(-2*pi*eta*b**4/3*( (diff(Curl(Lambda,gamma), rij[beta], rij[kappa2]) + diff(Curl(Lambda,beta), rij[gamma], rij[kappa2])) + 
                               (diff(Curl(kappa2,gamma), rij[beta], rij[Lambda]) + diff(Curl(kappa2,beta), rij[gamma], rij[Lambda]))))


def k1s2sSym(alpha, beta, gamma):
    return delG(alpha,gamma,beta) + delG(alpha, beta, gamma)

def K1s2s(alpha,beta,gamma):
    return simplify(-4*pi*eta*b**2/3*(k1s2sSym(alpha, beta, gamma) + Laplacian(k1s2sSym(alpha, beta, gamma))))

def K1s3t(alpha,beta):
    return simplify(-6*pi*eta*b**3/25*lapG(alpha, beta))

def K2s3t(alpha, gamma1, nu2):
    return simplify(-3*pi*eta*b**4/25*(dellapG(alpha,nu2,gamma1) + dellapG(gamma1, nu2, alpha)))

def K3s3t(alpha, gamma1, gamma2, nu2):
    return simplify(-6*pi*eta*b**5/25*Laplacian(g3s1Sym(alpha,gamma1,gamma2,nu2)))


def G2s3t(alpha,gamma1,nu2):
    return simplify(-b**3/10*(dellapG(alpha,nu2,gamma1) + dellapG(gamma1,nu2,alpha)))

def G3s3t(alpha,gamma1,gamma2,nu2):
    return simplify(-b**4/5*Laplacian(g3s1Sym(alpha,gamma1,gamma2,nu2)))


def G2s3a(alpha,gamma1,mu,nu2): #sign change -> eps(mu,nu1,beta)
    return simplify(b**3/3*( diff(Curl(mu,alpha), rij[gamma1], rij[nu2]) - 1/3*diff(summation(eps(mu,nu2,beta)*Laplacian(G(alpha,beta)), (beta,0,2)), rij[gamma1])
                   + diff(Curl(mu,gamma1), rij[alpha], rij[nu2]) - 1/3*diff(summation(eps(mu,nu2,beta)*Laplacian(G(gamma1,beta)), (beta,0,2)), rij[alpha])))

def G3a3a(Lambda, kappa2, mu, nu2): #sign change -> eps(mu,nu1,beta)
    return simplify(b**4/3*(diff(CurlCurl(mu,Lambda), rij[nu2], rij[kappa2]) + diff(CurlCurl(mu,kappa2), rij[nu2], rij[Lambda])))


def G3s3a(alpha,gamma1,gamma2,mu,nu2): #sign change -> eps(mu,nu1,beta)
    return simplify(2*b**4/3*(diff(g3sCurlSym(alpha,gamma1,gamma2,mu), rij[nu2]) - 1/3*Laplacian(summation(eps(mu,nu2,beta)*g3s1Sym(alpha,gamma1,gamma2,beta), (beta,0,2)))))


def g2s3sSym(alpha,gamma1,kappa2, kappa1, beta):
    return diff(g13sSym(alpha,kappa2,kappa1,beta), rij[gamma1]) + diff(g13sSym(gamma1, kappa2, kappa1, beta), rij[alpha])

def G2s3s(alpha,gamma1,kappa2, kappa1, beta):
    return simplify(0.5*b**3*(g2s3sSym(alpha,gamma1,kappa2, kappa1, beta) + 6*b**2/35*Laplacian(g2s3sSym(alpha,gamma1,kappa2, kappa1, beta))))


def G3a3s(Lambda, eta2, kappa2, kappa1, beta):
    return simplify(0.5*b**4*(diff(g3sCurlSym(beta, kappa1, kappa2, Lambda), rij[eta2]) + diff(g3sCurlSym(beta, kappa1, kappa2, eta2), rij[Lambda])))


def deldelg13sSym(alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return diff(g13sSym(alpha,kappa2,kappa1,beta), rij[gamma1], rij[gamma2])

def lapg13sSym(alpha,kappa2,kappa1,beta):
    return Laplacian(g13sSym(alpha,kappa2,kappa1,beta))

def g3s3sSym(alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return 1/3*(deldelg13sSym(alpha,gamma1,gamma2,kappa2,kappa1,beta) + deldelg13sSym(gamma1,gamma2,alpha,kappa2,kappa1,beta) + 
                deldelg13sSym(gamma2,alpha,gamma1,kappa2,kappa1,beta)) - 1/15*(
        KroneckerDelta[gamma1,gamma2]*lapg13sSym(alpha, kappa2, kappa1, beta) + 
        KroneckerDelta[gamma2,alpha]*lapg13sSym(gamma1, kappa2, kappa1, beta) + 
        KroneckerDelta[alpha,gamma1]*lapg13sSym(gamma2, kappa2, kappa1, beta))

def G3s3s(alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return simplify(b**4*(g3s3sSym(alpha,gamma1,gamma2,kappa2,kappa1,beta) + b**2/7*Laplacian(g3s3sSym(alpha,gamma1,gamma2,kappa2,kappa1,beta))))


def G3a2s(Lambda, kappa2, kappa1, beta):
    return simplify(-0.25*b**3*(diff(Curl(Lambda,beta), rij[kappa1], rij[kappa2]) + diff(Curl(Lambda,kappa1), rij[beta], rij[kappa2])
                       + diff(Curl(kappa2,beta), rij[kappa1], rij[Lambda]) + diff(Curl(kappa2,kappa1), rij[beta], rij[Lambda])))


def g3s2sSym(alpha,gamma1,gamma2, kappa1, beta):
    return diff(g3s1Sym(alpha,gamma1,gamma2,beta), rij[kappa1]) + diff(g3s1Sym(alpha,gamma1,gamma2,kappa1), rij[beta])

def G3s2s(alpha,gamma1,gamma2,kappa1,beta):
    return simplify(-0.5*b**3*(g3s2sSym(alpha,gamma1,gamma2, kappa1, beta) + 6*b**2/35*Laplacian(g3s2sSym(alpha,gamma1,gamma2, kappa1, beta))))


def G2a3a(Lambda, mu, nu2): # sign change -> eps(mu,nu1,beta)
    return simplify(2*b**3/3*diff(CurlCurl(Lambda, mu), rij[nu2]))


def G2a3s(Lambda,kappa2,kappa1,beta):
    return simplify(b**3*g3sCurlSym(beta, kappa1, kappa2, Lambda))

def K2a2s(Lambda, beta, gamma):
    return simplify(-4*pi*eta*b**3/3*(diff(Curl(Lambda,gamma), rij[beta]) + diff(Curl(Lambda,beta), rij[gamma])) )

In [8]:
## zero elements

def G3t1s(alpha,beta):
    return 0

def G3t2a(alpha,beta):
    return 0

def G3t2s(alpha, beta, kappa1):
    return 0

def G3t3t(alpha,beta):
    return 0

def G3t3a(alpha, beta, kappa1):
    return 0

def G3t3s(alpha, beta, kappa1, kappa2):
    return 0

def K3t2s(alpha, beta, kappa1):
    return 0

def K3t3t(alpha,beta):
    return 0

def K3t3a(alpha, beta, kappa1):
    return 0

def K3t3s(alpha, beta, kappa1, kappa2):
    return 0

def G2a3t(alpha,beta):
    return 0

def G3a3t(alpha, gamma1, beta):
    return 0

def K2a3t(alpha,beta):
    return 0

def K3a3t(alpha, gamma1, beta):
    return 0

### Matrix assembly

In [9]:
n_jobs = -1

In [10]:
modes = ['1s', '2a', '2s', '3t', '3a', '3s']
modesV = ['2s', '3t'] ##possible slip modes, for double-layer matrix elements
SLDL  = ['G', 'K']
dim   = {'1s':3, '2a':3, '2s':3*3, '3t':3, '3a':3*3, '3s':3*3*3}

In [ ]:
%%time
for sldl in SLDL:
    if sldl =='G':
        for mode1 in modes:
            for mode2 in modes:
                if mode1 in ['1s', '2a', '3t']:
                    if mode2 in ['1s', '2a', '3t']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, kj) for ki in range(3) 
                                                                            for kj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 in ['2s', '3a']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, kj, lj) for ki in range(3)
                                                                                for kj in range(3)
                                                                                    for lj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 =='3s':
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, kj, lj, mj) for ki in range(3)
                                                                                    for kj in range(3)
                                                                                        for lj in range(3)
                                                                                            for mj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')



                elif mode1 in ['2s', '3a']:
                    if mode2 in ['1s', '2a', '3t']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, li, kj) for ki in range(3) 
                                                                                for li in range(3)
                                                                                    for kj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 in ['2s', '3a']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, li, kj, lj) for ki in range(3)
                                                                                    for li in range(3)
                                                                                        for kj in range(3)
                                                                                            for lj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 =='3s':
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, li, kj, lj, mj) for ki in range(3)
                                                                                        for li in range(3)
                                                                                            for kj in range(3)
                                                                                                for lj in range(3)
                                                                                                    for mj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')



                elif mode1 == '3s':
                    if mode2 in ['1s', '2a', '3t']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, li, mi, kj) for ki in range(3) 
                                                                                    for li in range(3)
                                                                                        for mi in range(3)
                                                                                            for kj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 in ['2s', '3a']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, li, mi, kj, lj) for ki in range(3)
                                                                                        for li in range(3)
                                                                                            for mi in range(3)
                                                                                                for kj in range(3)
                                                                                                    for lj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 =='3s':
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, li, mi, kj, lj, mj) for ki in range(3)
                                                                                            for li in range(3)
                                                                                                for mi in range(3)
                                                                                                    for kj in range(3)
                                                                                                        for lj in range(3)
                                                                                                            for mj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')
    
    
    
    elif sldl == 'K':
        for mode1 in modes:
            for mode2 in modesV:
                if mode1 in ['1s', '2a', '3t']:
                    if mode2 in ['1s', '2a', '3t']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, kj) for ki in range(3) 
                                                                            for kj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 in ['2s', '3a']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, kj, lj) for ki in range(3)
                                                                                for kj in range(3)
                                                                                    for lj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 =='3s':
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, kj, lj, mj) for ki in range(3)
                                                                                    for kj in range(3)
                                                                                        for lj in range(3)
                                                                                            for mj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')



                elif mode1 in ['2s', '3a']:
                    if mode2 in ['1s', '2a', '3t']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, li, kj) for ki in range(3) 
                                                                                for li in range(3)
                                                                                    for kj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 in ['2s', '3a']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, li, kj, lj) for ki in range(3)
                                                                                    for li in range(3)
                                                                                        for kj in range(3)
                                                                                            for lj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 =='3s':
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, li, kj, lj, mj) for ki in range(3)
                                                                                        for li in range(3)
                                                                                            for kj in range(3)
                                                                                                for lj in range(3)
                                                                                                    for mj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')



                elif mode1 == '3s':
                    if mode2 in ['1s', '2a', '3t']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, li, mi, kj) for ki in range(3) 
                                                                                    for li in range(3)
                                                                                        for mi in range(3)
                                                                                            for kj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 in ['2s', '3a']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, li, mi, kj, lj) for ki in range(3)
                                                                                        for li in range(3)
                                                                                            for mi in range(3)
                                                                                                for kj in range(3)
                                                                                                    for lj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 =='3s':
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, li, mi, kj, lj, mj) for ki in range(3)
                                                                                            for li in range(3)
                                                                                                for mi in range(3)
                                                                                                    for kj in range(3)
                                                                                                        for lj in range(3)
                                                                                                            for mj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

G1s1s done
G1s2a done
G1s2s done
G1s3t done
G1s3a done
G1s3s done
G2a1s done
G2a2a done
G2a2s done
G2a3t done
G2a3a done
G2a3s done
G2s1s done
G2s2a done
G2s2s done
G2s3t done
G2s3a done
G2s3s done
G3t1s done
G3t2a done
G3t2s done
G3t3t done
G3t3a done
G3t3s done
G3a1s done
G3a2a done
G3a2s done
G3a3t done
G3a3a done
G3a3s done
G3s1s done
G3s2a done
G3s2s done
G3s3t done
G3s3a done


In [ ]:
## In python code will be force[i,0], so have to create force as np.zeros([3*Np,1]) object instead of np.zeros([3*Np])
force  = Matrix(MatrixSymbol('force',3,1)) ##create an actual Matrix from it, have to pass to 'Matrix' 
torque = Matrix(MatrixSymbol('torque',3,1))
VH     = Matrix(MatrixSymbol('VH',48,1))
FH     = Matrix(MatrixSymbol('FH',48,1))

Create block matrices, then multiply with vectors and then print as functions

In [ ]:
## need numpy diagonals and convert them to Matrix object for GoHH and KoHH

In [ ]:
matG1sH = Matrix(BlockMatrix([matG1s2s, matG1s3t, matG1s3a, matG1s3s]))
matG2aH = Matrix(BlockMatrix([matG2a2s, matG2a3t, matG2a3a, matG2a3s]))

matK1sH = Matrix(BlockMatrix([matK1s2s, matK1s3t, Matrix(ZeroMatrix(3,36))]))
matK2aH = Matrix(BlockMatrix([matK2a2s, matK2a3t, Matrix(ZeroMatrix(3,36))]))

matGH1s = Matrix(BlockMatrix([[matG2s1s],
                              [matG3t1s],
                              [matG3a1s],
                              [matG3s1s]]))
matGH2a = Matrix(BlockMatrix([[matG2s2a],
                              [matG3t2a],
                              [matG3a2a],
                              [matG3s2a]]))

matGHH = Matrix(BlockMatrix([[matG2s2s, matG2s3t, matG2s3a, matG2s3s],
                             [matG3t2s, matG3t3t, matG3t3a, matG3t3s],
                             [matG3a2s, matG3a3t, matG3a3a, matG3a3s],
                             [matG3s2s, matG3s3t, matG3s3a, matG3s3s]]))

matKHH_nonzero = Matrix(BlockMatrix([[matK2s2s, matK2s3t],
                                     [matK3t2s, matK3t3t],
                                     [matK3a2s, matK3a3t],
                                     [matK3s2s, matK3s3t]]))
matKHH = Matrix(BlockMatrix([matKHH_nonzero, Matrix(ZeroMatrix(48,36))]))

##diagonal elements
#g1s = 1/(6*pi*eta*b)
#g2a = 1/(4*pi*eta*b)

g2s = 3/(20*pi*eta*b)
g3t = 1/(2*pi*eta*b)
g3a = 3/(2*pi*eta*b)
g3s = 6/(7*pi*eta*b)

arrayGoHH = np.diag(np.block([np.full(9, g2s),
                              np.full(3, g3t),
                              np.full(9, g3a),
                              np.full(27, g3s)]))

matGoHH = Matrix(arrayGoHH)

halfMinusk2s = 0.6
halfMinusk3t = 0.4
halfMinusk3a = 0.8
halfMinusk3s = 19/35

arrayKoHH = np.diag(np.block([np.full(9, halfMinusk2s),
                              np.full(3, halfMinusk3t),
                              np.full(9, halfMinusk3a),
                              np.full(27, halfMinusk3s)]))

matKoHH = Matrix(arrayKoHH)

Create vectors instead of constructing the full matrix

In [ ]:
vecGH1sF = matGH1s*force
vecGH2aT = matGH2a*torque

vecKoHHVH = matKoHH*VH
vecKHHVH = matKHH*VH

vecGoHHFH = matGoHH*FH
vecGHHFH = matGHH*FH

vecG1s1sF = matG1s1s*force
vecG2a1sF = matG2a1s*force

vecG1s2aT = matG1s2a*torque
vecG2a2aT = matG2a2a*torque

vecG1sHFH = matG1sH*FH
vecG2aHFH = matG2aH*FH

vecK1sHVH = matK1sH*VH
vecK2aHVH = matK2aH*VH

_____________

### Write symbolic results to file as reusable python functions

In [ ]:
ME_file = "fullME.py"

with open(ME_file, "w") as text_file:
    print("import numpy", file=text_file)
    print("PI = 3.14159265359\n", file=text_file)
    
    print("def G1s1sF(xij,yij,zij, b,eta, force):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG1s1sF),(3,)).tolist()), file=text_file)
    print("def G2a1sF(xij,yij,zij, b,eta, force):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2a1sF),(3,)).tolist()), file=text_file)
    
    print("def G1s2aT(xij,yij,zij, b,eta, torque):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG1s2aT),(3,)).tolist()), file=text_file)
    print("def G2a2aT(xij,yij,zij, b,eta, torque):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2a2aT),(3,)).tolist()), file=text_file)
    
    print("def GH1sF(gh1sf, dimH, i, xij,yij,zij, b,eta, force):", file=text_file)
    print("    gh1sf[dimH*i:dimH*(i+1)] += {}".format(np.reshape(np.asarray(vecGH1sF),(48,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GH2aT(gh2at, dimH, i, xij,yij,zij, b,eta, torque):", file=text_file)
    print("    gh2at[dimH*i:dimH*(i+1)] += {}".format(np.reshape(np.asarray(vecGH2aT),(48,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def KoHHVH(khhvh, dimH, i, b,eta, VH):", file=text_file)
    print("    khhvh[dimH*i:dimH*(i+1)] -= {}".format(np.reshape(np.asarray(vecKoHHVH),(48,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def KHHVH(khhvh, dimH, i, xij,yij,zij, b,eta, VH):", file=text_file)
    print("    khhvh[dimH*i:dimH*(i+1)] += {}".format(np.reshape(np.asarray(vecKHHVH),(48,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GoHHFH(ghhfh, dimH, i, b,eta, FH):", file=text_file)
    print("    ghhfh[dimH*i:dimH*(i+1)] += {}".format(np.reshape(np.asarray(vecGoHHFH),(48,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GHHFH(ghhfh, dimH, i, xij,yij,zij, b,eta, FH):", file=text_file)
    print("    ghhfh[dimH*i:dimH*(i+1)] += {}".format(np.reshape(np.asarray(vecGHHFH),(48,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def G1sHFH(xij,yij,zij, b,eta, FH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG1sHFH),(3,)).tolist()), file=text_file)
    print("def G2aHFH(xij,yij,zij, b,eta, FH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2aHFH),(3,)).tolist()), file=text_file)
    
    print("def K1sHVH(xij,yij,zij, b,eta, VH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecK1sHVH),(3,)).tolist()), file=text_file)
    print("def K2aHVH(xij,yij,zij, b,eta, VH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecK2aHVH),(3,)).tolist()), file=text_file)
    
    
    
## replace things for easier usage with python     
with open(ME_file, 'r') as file:
    filedata = file.read()
    
filedata = filedata.replace('pi', 'PI')
filedata = filedata.replace('sqrt', 'numpy.sqrt')

filedata = re.sub("force\[(\d), 0\]", r"force[\1]", filedata)
filedata = re.sub("torque\[(\d), 0\]", r"torque[\1]", filedata)
filedata = re.sub("VH\[(\d||\d\d), 0\]", r"VH[\1]", filedata)
filedata = re.sub("FH\[(\d||\d\d), 0\]", r"FH[\1]", filedata)

with open(ME_file, 'w') as file:
    file.write(filedata)

______________________________________
_______________________

_____________

### Save **matrix** expressions for **direct** solver 

This is to eliminate one variable in the uncertainty whether the results are correct/incorrect. Integrator + Krylov seems very touchy

In [17]:
ME_file = "matrixME.py"

with open(ME_file, "w") as text_file:
    print("import numpy", file=text_file)
    print("PI = 3.14159265359\n", file=text_file)
    
    print("def G1s1s(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG1s1s),(3,3)).tolist()), file=text_file)
    print("def G2a1s(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG2a1s),(3,3)).tolist()), file=text_file)
    
    print("def G1s2a(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG1s2a),(3,3)).tolist()), file=text_file)
    print("def G2a2a(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG2a2a),(3,3)).tolist()), file=text_file)
    
    print("def GH1s(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGH1s),(25,3)).tolist()), file=text_file)
    print("def GH2a(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGH2a),(25,3)).tolist()), file=text_file)
    
    print("def KoHH(b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matKoHH),(25,25)).tolist()), file=text_file)
    print("def KHH(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matKHH),(25,25)).tolist()), file=text_file)
    
    print("def GoHH(b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGoHH),(25,25)).tolist()), file=text_file)
    print("def GHH(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGHH),(25,25)).tolist()), file=text_file)
    
    print("def G1sH(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG1sH),(3,25)).tolist()), file=text_file)
    print("def G2aH(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG2aH),(3,25)).tolist()), file=text_file)
    
    print("def K1sH(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matK1sH),(3,25)).tolist()), file=text_file)
    print("def K2aH(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matK2aH),(3,25)).tolist()), file=text_file)
    
    
    
## replace things for easier usage with python     
with open(ME_file, 'r') as file:
    filedata = file.read()
    
filedata = filedata.replace('pi', 'PI')
filedata = filedata.replace('sqrt', 'numpy.sqrt')

with open(ME_file, 'w') as file:
    file.write(filedata)

______________________________________
_______________________

In [22]:
matK3a2s[0,1].subs({xij:0,yij:0, zij:2.3})/2

0.0357345778495646*b**4